In [1]:
!pip install akinator.py

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import random
import import_ipynb
from question_answering import *
from copy import deepcopy

import akinator

importing Jupyter notebook from question_answering.ipynb


[nltk_data] Downloading package punkt to /home/agupta44/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/agupta44/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/agupta44/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/agupta44/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Defaulting to user installation because normal site-packages is not writeable
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 1.13.3 requires huggingface-hub<0.1.0,>=0.0.19, but you have huggingface-hub 0.0.12 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
  Using cached huggingface_hub-0.0.19-py3-none-any.whl (56 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.12
    Uninstalling huggingface-hub-0.0.12:
    

Reusing dataset super_glue (/home/agupta44/.cache/huggingface/datasets/super_glue/boolq/1.0.2/2fb163bca9085c1deb906aff20f00c242227ff704a4e8c9cfdfe820be3abfc83)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/agupta44/.cache/huggingface/datasets/super_glue/boolq/1.0.2/2fb163bca9085c1deb906aff20f00c242227ff704a4e8c9cfdfe820be3abfc83/cache-85953c194f136f47.arrow
Loading cached processed dataset at /home/agupta44/.cache/huggingface/datasets/super_glue/boolq/1.0.2/2fb163bca9085c1deb906aff20f00c242227ff704a4e8c9cfdfe820be3abfc83/cache-2f6ec22aeaa03199.arrow
Loading cached processed dataset at /home/agupta44/.cache/huggingface/datasets/super_glue/boolq/1.0.2/2fb163bca9085c1deb906aff20f00c242227ff704a4e8c9cfdfe820be3abfc83/cache-44f50d16a63eceb6.arrow


ValueError: Shape of passed values is (9427, 2), indices imply (9427, 3)

In [4]:
from question_answering import get_data_from_file, question2answer

In [22]:
def play_baseline_game(guess, context_filename):
    aki = akinator.Akinator()
    data, paragraphs = get_data_from_file(context_filename)
    choices = ["yes", "no", "idk", "probably", "probably not"]
    choice_prob = [0.425, 0.425, 0.05, 0.05, 0.05]
    
    q = aki.start_game(language="en_animals")
    prob_guess = 0
    feature_count = 0
    
    while aki.progression <= 80:
        #a = random.choices(choices, weights=choice_prob, k=1)[0]
        #a = input(q + "\n\t")
        aug_guess = "a "+guess
        question = aki.question
        text_answer = question2answer(question.replace("your animal", aug_guess), data, paragraphs)
        print(text_answer)
        a = input(question.replace("your animal", aug_guess) + "\n\t")
        print("Question: ", aki.question)
        print("Answer: ", a)
        q = aki.answer(a)
        prev_aki = deepcopy(aki)
        prev_aki.win()
        print(prev_aki.guesses)
        for g in prev_aki.guesses:
            if g['name'] in guess or guess in g['name']:
                prob_guess += float(g['proba'])
                feature_count += 1
                break
    aki.win()
    
    if feature_count != 0:
        print("Avg probability: ", 0)
    else:
        print("Avg probability: ", prob_guess/feature_count)
    print("I guess", aki.first_guess['name'])
    if guess in aki.first_guess['name']:
        print("Yay\n")
    else:
        print("Oof\n")

In [6]:
def play_improved_game(guess, context_filename):
    data, paragraphs = get_data_from_file(context_filename)
    
    aki = akinator.Akinator()
    choices = ["yes", "no", "idk", "probably", "probably not"]
    choice_prob = [0.425, 0.425, 0.05, 0.05, 0.05]
    
    q = aki.start_game()
    prob_guess = 0
    feature_count = 1
    
    while aki.progression <= 80:
        print("Question: ", aki.question)
        
        text_answer, text_answer_score = question2answer(question, context)
        tf_answer = answer2tf(question, context, text_answer)
        a = tfanswer2choices(tf_answer, choices)
        
        q = aki.answer(a)
        prev_aki = deepcopy(aki)
        prev_aki.win()
        print(prev_aki.guesses)
        for g in prev_aki.guesses:
            if g['name'] == guess:
                prob_guess += float(g['proba'])
                feature_count += 1
                break
    aki.win()
    
    print("Avg probability: ", prob_guess/feature_count)
    print("I guess", aki.first_guess['name'])
    if guess in aki.first_guess['name']:
        print("Yay\n")
    else:
        print("Oof\n")

In [18]:
play_baseline_game("cheetah", "cheetah.txt")

{'score': 0.5548820495605469, 'start': 12418, 'end': 12426, 'answer': 'breeding'}
['The diploid number of chromosomes in the cheetah is 38, the same as in most other felids.[45] The cheetah was the first felid observed to have unusually low genetic variability among individuals,[46] which has led to poor breeding in captivity, increased spermatozoal defects, high juvenile mortality and increased susceptibility to diseases and infections.[47][48] A prominent instance was the deadly feline coronavirus outbreak in a cheetah breeding facility of Oregon in 1983 which had a mortality rate of 60%—higher than that recorded for previous epizootics of feline infectious peritonitis in any felid.[49] The remarkable homogeneity in cheetah genes has been demonstrated by experiments involving the major histocompatibility complex (MHC); unless the MHC genes are highly homogeneous in a population, skin grafts exchanged between a pair of unrelated individuals would be rejected. Skin grafts exchanged bet

{'score': 0.7953176498413086, 'start': 22417, 'end': 22452, 'answer': 'The cheetah has a total of 30 teeth'}
['The cheetah has a total of 30 teeth; the dental formula is']
Does your animal have sharp teeth?
	idk
Question:  Does your animal have sharp teeth?
Answer:  idk
[{'id': '524', 'name': 'an Asian elephant', 'id_base': '154313', 'proba': '0.0243159', 'description': '---', 'valide_contrainte': '1', 'ranking': '40', 'pseudo': 'elokence_photo', 'picture_path': '1762185492__510143789.jpeg', 'corrupt': '0', 'relative': '0', 'award_id': '-1', 'flag_photo': 0, 'absolute_picture_path': 'https://photos.clarinea.fr/BL_43_en/600/1762185492__510143789.jpeg'}, {'id': '41', 'name': 'a panda', 'id_base': '1144', 'proba': '0.0240437', 'description': '---', 'valide_contrainte': '1', 'ranking': '30', 'pseudo': 'elokence_photo', 'picture_path': 'Fotolia_44411640_XS.jpg', 'corrupt': '0', 'relative': '0', 'award_id': '-1', 'flag_photo': 0, 'absolute_picture_path': 'https://photos.clarinea.fr/BL_43_en/

{'score': 0.5554339289665222, 'start': 51089, 'end': 51166, 'answer': 'cheetah populations across the region have become smaller and more fragmented'}
['The first survey of cheetah populations in Africa by Norman Myers in 1975 estimated a population of 15,000 individuals throughout Sub-Saharan Africa. The range covered most of eastern and southern Africa, except for the desert region on the western coast of modern-day Angola and Namibia.[135] In the following years, as their natural habitat has been modified dramatically, cheetah populations across the region have become smaller and more fragmented.[136]']
Is your animal is smaller than a human?
	idk
Question:  Is your animal is smaller than a human?
Answer:  idk
[{'id': '17870', 'name': 'a Giant Panda', 'id_base': '178007', 'proba': '0.099865', 'description': '---', 'valide_contrainte': '1', 'ranking': '90', 'pseudo': 'Unknown', 'picture_path': 'partenaire/x/178007__1983474382.jpg', 'corrupt': '0', 'relative': '0', 'award_id': '-1', '

Does your animal have paws?
	yes
Question:  Does your animal have paws?
Answer:  yes
[{'id': '17870', 'name': 'a Giant Panda', 'id_base': '178007', 'proba': '0.170771', 'description': '---', 'valide_contrainte': '1', 'ranking': '90', 'pseudo': 'Unknown', 'picture_path': 'partenaire/x/178007__1983474382.jpg', 'corrupt': '0', 'relative': '0', 'award_id': '-1', 'flag_photo': 0, 'absolute_picture_path': 'https://photos.clarinea.fr/BL_43_en/600/partenaire/x/178007__1983474382.jpg'}, {'id': '41', 'name': 'a panda', 'id_base': '1144', 'proba': '0.0859941', 'description': '---', 'valide_contrainte': '1', 'ranking': '30', 'pseudo': 'elokence_photo', 'picture_path': 'Fotolia_44411640_XS.jpg', 'corrupt': '0', 'relative': '0', 'award_id': '-1', 'flag_photo': 0, 'absolute_picture_path': 'https://photos.clarinea.fr/BL_43_en/Fotolia_44411640_XS.jpg'}, {'id': '493', 'name': 'a Royal Bengal tiger', 'id_base': '152716', 'proba': '0.0417246', 'description': '---', 'valide_contrainte': '1', 'ranking': '16

{'score': 0.5153712034225464, 'start': 18426, 'end': 18446, 'answer': 'cheetah hunts mainly'}
["The head is small and more rounded compared to the big cats.[68] Saharan cheetahs have canine-like slim faces.[65] The ears are small, short and rounded; they are tawny at the base and on the edges and marked with black patches on the back. The eyes are set high and have round pupils.[62] The whiskers, shorter and fewer than those of other felids, are fine and inconspicuous.[69] The pronounced tear streaks (or malar stripes), unique to the cheetah, originate from the corners of the eyes and run down the nose to the mouth. The role of these streaks is not well understood—they may protect the eyes from the sun's glare (a helpful feature as the cheetah hunts mainly during the day), or they could be used to define facial expressions.[65] The exceptionally long and muscular tail, with a bushy white tuft at the end, measures 60–80 cm (24–31 in).[70] While the first two-thirds of the tail are cover

{'score': 0.5075576305389404, 'start': 19067, 'end': 19105, 'answer': 'spots fuse to form stripes on the back'}
['The cheetah is superficially similar to the leopard, but the leopard has rosettes instead of spots and lacks tear streaks.[63][71] Moreover, the cheetah is slightly taller than the leopard. The serval resembles the cheetah in physical build, but is significantly smaller, has a shorter tail and its spots fuse to form stripes on the back.[72] The cheetah appears to have evolved convergently with canids in morphology and behaviour; it has canine-like features such as a relatively long snout, long legs, a deep chest, tough paw pads and blunt, semi-retractable claws.[73][74] The cheetah has often been likened to the greyhound, as both have similar morphology and the ability to reach tremendous speeds in a shorter time than other mammals,[67][70] but the cheetah can attain higher maximum speeds.[75]']
Has your animal got spots?
	yes
Question:  Has your animal got spots?
Answer:  

Is your animal native to Egypt?
	idk
Question:  Is your animal native to Egypt?
Answer:  idk
[{'id': '4874', 'name': 'a king cheetah', 'id_base': '160851', 'proba': '0.535429', 'description': '---', 'valide_contrainte': '1', 'ranking': '111', 'pseudo': 'MEH_44', 'picture_path': 'partenaire/f/160851__784343264.jpg', 'corrupt': '0', 'relative': '0', 'award_id': '-1', 'flag_photo': 0, 'absolute_picture_path': 'https://photos.clarinea.fr/BL_43_en/600/partenaire/f/160851__784343264.jpg'}, {'id': '55', 'name': 'a cheetah', 'id_base': '1759', 'proba': '0.168528', 'description': '---', 'valide_contrainte': '1', 'ranking': '21', 'pseudo': 'elokence_photo', 'picture_path': 'Fotolia_34598898_XS.jpg', 'corrupt': '0', 'relative': '0', 'award_id': '-1', 'flag_photo': 0, 'absolute_picture_path': 'https://photos.clarinea.fr/BL_43_en/Fotolia_34598898_XS.jpg'}, {'id': '493', 'name': 'a Royal Bengal tiger', 'id_base': '152716', 'proba': '0.0618191', 'description': '---', 'valide_contrainte': '1', 'rankin

{'score': 0.19517891108989716, 'start': 28695, 'end': 28721, 'answer': 'Some males are territorial'}
['Some males are territorial, and group together for life, forming coalitions that collectively defend a territory which ensures maximum access to females—this is unlike the behaviour of the male lion who mates with a particular group (pride) of females. In most cases, a coalition will consist of brothers born in the same litter who stayed together after weaning, but biologically unrelated males are often allowed into the group; in the Serengeti 30% members in coalitions are unrelated males.[65] If a cub is the only male in a litter he will typically join an existing group, or form a small group of solitary males with two or three other lone males who may or may not be territorial. In the Kalahari Desert around 40% of the males live in solitude.[62][65]']
Is your animal considered territorial?
	yes
Question:  Is your animal considered territorial?
Answer:  yes
[{'id': '55', 'name': 'a c

{'score': 0.2043295055627823, 'start': 19040, 'end': 19105, 'answer': 'has a shorter tail and its spots fuse to form stripes on the back'}
['The cheetah is superficially similar to the leopard, but the leopard has rosettes instead of spots and lacks tear streaks.[63][71] Moreover, the cheetah is slightly taller than the leopard. The serval resembles the cheetah in physical build, but is significantly smaller, has a shorter tail and its spots fuse to form stripes on the back.[72] The cheetah appears to have evolved convergently with canids in morphology and behaviour; it has canine-like features such as a relatively long snout, long legs, a deep chest, tough paw pads and blunt, semi-retractable claws.[73][74] The cheetah has often been likened to the greyhound, as both have similar morphology and the ability to reach tremendous speeds in a shorter time than other mammals,[67][70] but the cheetah can attain higher maximum speeds.[75]']
Has your animal got stripes?
	yes
Question:  Has you

In [21]:
play_baseline_game("penguin", "penguin.txt")

{'score': 0.42053478956222534, 'start': 18233, 'end': 18293, 'answer': 'they are unique among living penguins by nesting in burrows.'}
['The genera Spheniscus and Eudyptula contain species with a mostly Subantarctic distribution centred on South America; some, however, range quite far northwards. They all lack carotenoid colouration and the former genus has a conspicuous banded head pattern; they are unique among living penguins by nesting in burrows. This group probably radiated eastwards with the Antarctic Circumpolar Current out of the ancestral range of modern penguins throughout the Chattian (Late Oligocene), starting approximately 28 mya.[26] While the two genera separated during this time, the present-day diversity is the result of a Pliocene radiation, taking place some 4–2 mya.[26]']
Does your animal have paws?
	idk
Question:  Does your animal have paws?
Answer:  idk
[{'id': '468', 'name': 'The incognito guy', 'id_base': '152024', 'proba': '0.100279', 'description': '---', 'va

{'score': 0.2914234697818756, 'start': 27402, 'end': 27473, 'answer': 'most extant penguins are too small to survive in such cold environments'}
['Calculations of the heat loss and retention ability of marine endotherms [52] suggest that most extant penguins are too small to survive in such cold environments.[53] In 2007, Thomas and Fordyce wrote about the "heterothermic loophole" that penguins utilize in order to survive in Antarctica.[54] All extant penguins, even those that live in warmer climates, have a counter-current heat exchanger called the humeral plexus. The flippers of penguins have at least three branches of the axillary artery, which allows cold blood to be heated by blood that has already been warmed and limits heat loss from the flippers. This system allows penguins to efficiently use their body heat and explains why such small animals can survive in the extreme cold.[55]']
Is your animal is smaller than a human?
	yes
Question:  Is your animal is smaller than a human?
A

{'score': 0.00044818478636443615, 'start': 18255, 'end': 18293, 'answer': 'living penguins by nesting in burrows.'}
['The genera Spheniscus and Eudyptula contain species with a mostly Subantarctic distribution centred on South America; some, however, range quite far northwards. They all lack carotenoid colouration and the former genus has a conspicuous banded head pattern; they are unique among living penguins by nesting in burrows. This group probably radiated eastwards with the Antarctic Circumpolar Current out of the ancestral range of modern penguins throughout the Chattian (Late Oligocene), starting approximately 28 mya.[26] While the two genera separated during this time, the present-day diversity is the result of a Pliocene radiation, taking place some 4–2 mya.[26]']
Does your animal live in trees?
	no
Question:  Does your animal live in trees?
Answer:  no
[{'id': '589', 'name': 'a red-bellied black snake / Pseudechis porphyriacus', 'id_base': '154737', 'proba': '0.0503488', 'de

{'score': 0.7073720097541809, 'start': 28066, 'end': 28091, 'answer': 'They can drink salt water'}
['They can drink salt water because their supraorbital gland filters excess salt from the bloodstream.[56][57][58] The salt is excreted in a concentrated fluid from the nasal passages.']
Does your animal live in the water?
	idk
Question:  Does your animal live in the water?
Answer:  idk
[{'id': '589', 'name': 'a red-bellied black snake / Pseudechis porphyriacus', 'id_base': '154737', 'proba': '0.0834435', 'description': '---', 'valide_contrainte': '1', 'ranking': '30', 'pseudo': 'NOELLK', 'picture_path': 'partenaire/d/154737__518165535.jpg', 'corrupt': '1', 'relative': '0', 'award_id': '-1', 'flag_photo': 0, 'absolute_picture_path': 'https://photos.clarinea.fr/BL_43_en/600/partenaire/d/154737__518165535.jpg'}, {'id': '12', 'name': 'a snake', 'id_base': '133', 'proba': '0.0591072', 'description': '---', 'valide_contrainte': '1', 'ranking': '44', 'pseudo': 'elokence_photo', 'picture_path': 

{'score': 0.001517626689746976, 'start': 18233, 'end': 18270, 'answer': 'they are unique among living penguins'}
['The genera Spheniscus and Eudyptula contain species with a mostly Subantarctic distribution centred on South America; some, however, range quite far northwards. They all lack carotenoid colouration and the former genus has a conspicuous banded head pattern; they are unique among living penguins by nesting in burrows. This group probably radiated eastwards with the Antarctic Circumpolar Current out of the ancestral range of modern penguins throughout the Chattian (Late Oligocene), starting approximately 28 mya.[26] While the two genera separated during this time, the present-day diversity is the result of a Pliocene radiation, taking place some 4–2 mya.[26]']
Has your animal got horns?
	no
Question:  Has your animal got horns?
Answer:  no
[{'id': '589', 'name': 'a red-bellied black snake / Pseudechis porphyriacus', 'id_base': '154737', 'proba': '0.146467', 'description': '-

...

{'score': 0.17797234654426575, 'start': 18233, 'end': 18292, 'answer': 'they are unique among living penguins by nesting in burrows'}
['The genera Spheniscus and Eudyptula contain species with a mostly Subantarctic distribution centred on South America; some, however, range quite far northwards. They all lack carotenoid colouration and the former genus has a conspicuous banded head pattern; they are unique among living penguins by nesting in burrows. This group probably radiated eastwards with the Antarctic Circumpolar Current out of the ancestral range of modern penguins throughout the Chattian (Late Oligocene), starting approximately 28 mya.[26] While the two genera separated during this time, the present-day diversity is the result of a Pliocene radiation, taking place some 4–2 mya.[26]']
Does your animal live in the ground?
	yes
Question:  Does your animal live in the ground?
Answer:  yes
[{'id': '577', 'name': 'an angler / Lophius piscatorius', 'id_base': '154674', 'proba': '0.300

{'score': 0.30606573820114136, 'start': 1041, 'end': 1069, 'answer': 'The smallest penguin species'}
['The largest living species is the emperor penguin (Aptenodytes forsteri):[4] on average, adults are about 1.1 m (3 ft 7 in) tall and weigh 35 kg (77 lb). The smallest penguin species is the little blue penguin (Eudyptula minor), also known as the fairy penguin, which stands around 33 cm (13 in) tall and weighs 1 kg (2.2 lb).[5] In general today, larger penguins inhabit colder regions, and smaller penguins inhabit regions with temperate or tropical climates. Some prehistoric penguin species were enormous: as tall or heavy as an adult human. There was a great diversity of species in subantarctic regions, and at least one giant species in a region around 2,000 km south of the equator 35 mya, in a climate decidedly warmer than today.[which?]']
Is your animal smaller than a hamster?
	no
Question:  Is your animal smaller than a hamster?
Answer:  no
[{'id': '42', 'name': 'a platypus', 'id_ba

{'score': 0.032426416873931885, 'start': 18294, 'end': 18371, 'answer': 'This group probably radiated eastwards with the Antarctic Circumpolar Current'}
['The genera Spheniscus and Eudyptula contain species with a mostly Subantarctic distribution centred on South America; some, however, range quite far northwards. They all lack carotenoid colouration and the former genus has a conspicuous banded head pattern; they are unique among living penguins by nesting in burrows. This group probably radiated eastwards with the Antarctic Circumpolar Current out of the ancestral range of modern penguins throughout the Chattian (Late Oligocene), starting approximately 28 mya.[26] While the two genera separated during this time, the present-day diversity is the result of a Pliocene radiation, taking place some 4–2 mya.[26]']
Does your animal migrate long distances?
	no
Question:  Does your animal migrate long distances?
Answer:  no
[{'id': '1249', 'name': 'a duck-billed platypus', 'id_base': '155855'

{'score': 0.42768409848213196, 'start': 18242, 'end': 18270, 'answer': 'unique among living penguins'}
['The genera Spheniscus and Eudyptula contain species with a mostly Subantarctic distribution centred on South America; some, however, range quite far northwards. They all lack carotenoid colouration and the former genus has a conspicuous banded head pattern; they are unique among living penguins by nesting in burrows. This group probably radiated eastwards with the Antarctic Circumpolar Current out of the ancestral range of modern penguins throughout the Chattian (Late Oligocene), starting approximately 28 mya.[26] While the two genera separated during this time, the present-day diversity is the result of a Pliocene radiation, taking place some 4–2 mya.[26]']
Is your animal a mammal?
	idk
Question:  Is your animal a mammal?
Answer:  idk
[{'id': '6134', 'name': 'an emperor penguin', 'id_base': '162589', 'proba': '0.101009', 'description': '---', 'valide_contrainte': '1', 'ranking': '1

{'score': 0.030052049085497856, 'start': 505, 'end': 563, 'answer': 'half of their lives on land and the other half in the sea.'}
['Penguins (order Sphenisciformes /sfɪˈnɪsɪfɔːrmiːz/, family Spheniscidae /sfɪˈnɪsɪdiː/) are a group of aquatic flightless birds. They live almost exclusively in the southern hemisphere: only one species, the Galápagos penguin, is found north of the Equator. Highly adapted for life in the water, penguins have countershaded dark and white plumage and flippers for swimming. Most penguins feed on krill, fish, squid and other forms of sea life which they catch while swimming underwater. They spend roughly half of their lives on land and the other half in the sea.']
Can your animal live for many years?
	idk
Question:  Can your animal live for many years?
Answer:  idk
[{'id': '6134', 'name': 'an emperor penguin', 'id_base': '162589', 'proba': '0.71389', 'description': '---', 'valide_contrainte': '1', 'ranking': '157', 'pseudo': 'emperor penguin', 'picture_path': '

In [23]:
play_baseline_game("seagull", "seagull.txt")

{'score': 0.00798611156642437, 'start': 13726, 'end': 13760, 'answer': 'two in some of the smaller species'}
['Clutch size is typically three eggs, although it is two in some of the smaller species and only one egg for the swallow-tailed gull. Within colonies, birds synchronise their laying, with synchronisation being higher in larger colonies, although after a certain point, this levels off. The eggs of gulls are usually dark tan to brown or dark olive with dark splotches and scrawl markings, and are well camouflaged. Both sexes incubate the eggs, with incubation bouts lasting between one and four hours during the day and one parent incubating through the night.[13] Research on various bird species including the gull suggests that females form pair bonds with other females to obtain alloparental care for their dependent offspring, a behavior seen in other animal species, like the elephants, wolves, and the fathead minnow.[25]']
Is a seagull is smaller than a human?
	yes
Question:  Is 

{'score': 0.00029372869175858796, 'start': 10517, 'end': 10578, 'answer': "The gulls relied substantially on the Henslow's swimming crab"}
["Looking at the effect of humans on gull diet, overfishing of target prey such as sardines have caused a shift in diet and behavior. Analysis of yellow-legged gull's (Larus michahellis) pellets off the northwest coast of Spain revealed a shift from a sardine to crustacean-based diet.[22] This shift was linked to higher fishing efficiency and thus overall fish stock depletion.[22] Lastly, closure of nearby open-air landfills limited food availability for the gulls, furthering creating a stress on their shift in diet.[22] Between the years of 1974–1994, yellow-legged gull populations in Berlenga Island, Portugal, increased from 2600 to 44,698 individuals. Analyzing both adult and chick remains, researchers found a mixture of both natural prey and human refuse. The gulls relied substantially on the Henslow's swimming crab (Polybius henslowii). Yet, in

{'score': 0.07113953679800034, 'start': 2238, 'end': 2285, 'answer': 'Gulls have been observed preying on live whales'}
['Gulls nest in large, densely packed, noisy colonies. They lay two or three speckled eggs in nests composed of vegetation. The young are precocial, born with dark mottled down and mobile upon hatching.[7] Gulls are resourceful, inquisitive, and intelligent, the larger species in particular,[8] demonstrating complex methods of communication and a highly developed social structure. For example, many gull colonies display mobbing behavior, attacking and harassing predators and other intruders.[9] Certain species have exhibited tool-use behavior, such as the herring gull, using pieces of bread as bait with which to catch goldfish, for example.[10] Many species of gulls have learned to coexist successfully with humans and have thrived in human habitats.[11] Others rely on kleptoparasitism to get their food. Gulls have been observed preying on live whales, landing on the w

{'score': 0.20088647305965424, 'start': 15995, 'end': 16007, 'answer': 'White-winged'}
["White-winged gull is used to describe the four pale-winged, high Arctic-breeding taxa within the former group; these are Iceland gull, glaucous gull, Thayer's gull, and Kumlien's gull."]
Has a seagull got wings?
	yes
Question:  Has your animal got wings?
Answer:  yes
[{'id': '65', 'name': 'a butterfly', 'id_base': '2125', 'proba': '0.0930083', 'description': '---', 'valide_contrainte': '1', 'ranking': '114', 'pseudo': 'elokence_photo', 'picture_path': 'Fotolia_40503728_XS.jpg', 'corrupt': '0', 'relative': '0', 'award_id': '-1', 'flag_photo': 0, 'absolute_picture_path': 'https://photos.clarinea.fr/BL_43_en/600/Fotolia_40503728_XS.jpg'}, {'id': '3', 'name': 'a mosquito', 'id_base': '24', 'proba': '0.0481416', 'description': '---', 'valide_contrainte': '1', 'ranking': '188', 'pseudo': 'elokence_photo', 'picture_path': 'Fotolia_35871569_XS.jpg', 'corrupt': '0', 'relative': '0', 'award_id': '-1', 'flag_

Is it an insect?
	no
Question:  Is it an insect?
Answer:  no
[{'id': '178', 'name': 'a bat', 'id_base': '10687', 'proba': '0.15574', 'description': '---', 'valide_contrainte': '1', 'ranking': '118', 'pseudo': 'elokence_photo', 'picture_path': 'Fotolia_36441273_XS.jpg', 'corrupt': '0', 'relative': '0', 'award_id': '-1', 'flag_photo': 0, 'absolute_picture_path': 'https://photos.clarinea.fr/BL_43_en/600/Fotolia_36441273_XS.jpg'}, {'id': '135', 'name': 'a phoenix', 'id_base': '7671', 'proba': '0.0321487', 'description': '---', 'valide_contrainte': '1', 'ranking': '94', 'pseudo': 'elokence_photo', 'picture_path': 'Fotolia_42257880_XS.jpg', 'corrupt': '0', 'relative': '0', 'award_id': '-1', 'flag_photo': 0, 'absolute_picture_path': 'https://photos.clarinea.fr/BL_43_en/Fotolia_42257880_XS.jpg'}, {'id': '799', 'name': 'a vampire bat', 'id_base': '155235', 'proba': '0.0315271', 'description': '---', 'valide_contrainte': '1', 'ranking': '327', 'pseudo': 'X', 'picture_path': 'partenaire/h/155235_

...

Does a seagull represent something?
	no
Question:  Does your animal represent something?
Answer:  no
[{'id': '111', 'name': 'a buzzard', 'id_base': '5622', 'proba': '0.161491', 'description': '---', 'valide_contrainte': '1', 'ranking': '1967', 'pseudo': 'elokence_photo', 'picture_path': 'Fotolia_16501235_XS.jpg', 'corrupt': '0', 'relative': '0', 'award_id': '-1', 'flag_photo': 0, 'absolute_picture_path': 'https://photos.clarinea.fr/BL_43_en/600/Fotolia_16501235_XS.jpg'}, {'id': '491', 'name': 'a golden eagle', 'id_base': '152661', 'proba': '0.0481609', 'description': '---', 'valide_contrainte': '1', 'ranking': '122', 'pseudo': 'elokence_photo', 'picture_path': '1819754207__13604385.jpeg', 'corrupt': '0', 'relative': '0', 'award_id': '-1', 'flag_photo': 0, 'absolute_picture_path': 'https://photos.clarinea.fr/BL_43_en/1819754207__13604385.jpeg'}, {'id': '14087', 'name': 'white eagle', 'id_base': '173167', 'proba': '0.035652', 'description': 'Polish national bird', 'valide_contrainte': '1

{'score': 0.0010710058268159628, 'start': 2238, 'end': 2285, 'answer': 'Gulls have been observed preying on live whales'}
['Gulls nest in large, densely packed, noisy colonies. They lay two or three speckled eggs in nests composed of vegetation. The young are precocial, born with dark mottled down and mobile upon hatching.[7] Gulls are resourceful, inquisitive, and intelligent, the larger species in particular,[8] demonstrating complex methods of communication and a highly developed social structure. For example, many gull colonies display mobbing behavior, attacking and harassing predators and other intruders.[9] Certain species have exhibited tool-use behavior, such as the herring gull, using pieces of bread as bait with which to catch goldfish, for example.[10] Many species of gulls have learned to coexist successfully with humans and have thrived in human habitats.[11] Others rely on kleptoparasitism to get their food. Gulls have been observed preying on live whales, landing on the

{'score': 0.5979517102241516, 'start': 4204, 'end': 4236, 'answer': 'the ivory gull is entirely white'}
["The general pattern of plumage in adult gulls is a white body with a darker mantle; the extent to which the mantle is darker varies from pale grey to black. A few species vary in this, the ivory gull is entirely white, and some like the lava gull and Heermann's gull have partly or entirely grey bodies. The wingtips of most species are black, which improves their resistance to wear and tear, usually with a diagnostic pattern of white markings. The head of a gull may be covered by a dark hood or be entirely white. The plumage of the head varies by breeding season; in nonbreeding dark-hooded gulls, the hood is lost, sometimes leaving a single spot behind the eye, and in white-headed gulls, nonbreeding heads may have streaking.[13]"]
Is a seagull white?
	yes
Question:  Is your animal white?
Answer:  yes
[{'id': '14087', 'name': 'white eagle', 'id_base': '173167', 'proba': '0.146545', '

{'score': 0.22923392057418823, 'start': 3352, 'end': 3399, 'answer': 'The bill colour is often yellow with a red spot'}
["Gulls range in size from the little gull, at 120 grams (4+1⁄4 ounces) and 29 centimetres (11+1⁄2 inches), to the great black-backed gull, at 1.75 kg (3 lb 14 oz) and 76 cm (30 in). They are generally uniform in shape, with heavy bodies, long wings, and moderately long necks. The tails of all but three species are rounded; the exceptions being Sabine's gull and swallow-tailed gulls, which have forked tails, and Ross's gull, which has a wedge-shaped tail. Gulls have moderately long legs, especially when compared to the similar terns, with fully webbed feet. The bill is generally heavy and slightly hooked, with the larger species having stouter bills than the smaller species. The bill colour is often yellow with a red spot for the larger white-headed species and red, dark red or black in the smaller species.[13]"]
Is a seagull yellow?
	np
Question:  Is your animal yell

InvalidAnswerError: 
        You put "np", which is an invalid answer.
        The answer must be one of these:
            - "yes" OR "y" OR "0" for YES
            - "no" OR "n" OR "1" for NO
            - "i" OR "idk" OR "i dont know" OR "i don't know" OR "2" for I DON'T KNOW
            - "probably" OR "p" OR "3" for PROBABLY
            - "probably not" OR "pn" OR "4" for PROBABLY NOT
        